In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./data/LLCP2020.csv')

In [3]:
def invalid_col_val_msg(colname, val, additional=''):
    return f'Invalid value in {colname}: {val}{additional}'

In [4]:
def preprocess_DIABAGE3(df):
    df['DIABAGE3_numeric'] = df['DIABAGE3']
    df.loc[(df['DIABAGE3'] > 97.5) | (df['DIABAGE3'].isna()), 'DIABAGE3_numeric'] = -1.
    
    df.loc[(df['DIABAGE3'] > 0.) & (df['DIABAGE3'] < 97.5), 'DIABAGE3_categorical'] = 1
    df.loc[(df['DIABAGE3'] > 97.5) & (df['DIABAGE3'] < 98.5), 'DIABAGE3_categorical'] = 7
    df.loc[(df['DIABAGE3'] > 98.5) & (df['DIABAGE3'] < 99.5), 'DIABAGE3_categorical'] = 9
    df.loc[df['DIABAGE3'].isna(), 'DIABAGE3_categorical'] = 0
    df['DIABAGE3_categorical'] = df['DIABAGE3_categorical'].astype('category')
    return df

In [5]:
df = preprocess_DIABAGE3(df)

In [6]:
def convert_FEETCHK3_numeric(value):
    if 101 <= value <= 199:
        return value * 365
    elif 201 <= value <= 299: # Here, we assume there are 52 weeks in a year
        return value * 52
    elif 301 <= value <= 399:
        return value * 12
    elif 401 <= value <= 499:
        return value
    else:
        return -1

def convert_FEETCHK3_categorical(value):
    # 200, 300, 400 are not in the original
    # dataset, so we can assume the range of
    # valid FEETCHK3 in [101, 499]
    if np.isnan(value):
        return 0
    elif 101 <= value <= 499:
        return 1
    elif value == 555:
        return 5
    elif value == 777:
        return 7
    elif value == 888:
        return 8
    elif value == 999:
        return 9
    else:
        raise ValueError('Invalid value in FEETCHK3:', value)

In [7]:
df['FEETCHK3_numeric'] = df['FEETCHK3'].apply(convert_FEETCHK3_numeric)
df['FEETCHK3_categorical'] = df['FEETCHK3'].apply(convert_FEETCHK3_categorical).astype('category')

In [8]:
df[['FEETCHK3', 'FEETCHK3_numeric', 'FEETCHK3_categorical']]

,FEETCHK3,FEETCHK3_numeric,FEETCHK3_categorical
0,NaN,-1.0,0
1,NaN,-1.0,0
2,NaN,-1.0,0
3,NaN,-1.0,0
4,NaN,-1.0,0
...,...,...,...
401953,NaN,-1.0,0
401954,NaN,-1.0,0
401955,NaN,-1.0,0
401956,NaN,-1.0,0


In [9]:
def convert_CHILDREN_numeric(n_children):
    # Note:
    # There is no zero children case
    # df.loc[df['CHILDREN'] == 0] : 0 rows
    if 0 <= n_children <= 87:
        return n_children
    else:
        return -1
    
def convert_CHILDREN_categorical(n_children):
    if np.isnan(n_children):
        return 0
    elif 1 <= n_children <= 87:
        return 1
    elif n_children == 88:
        return 8
    elif n_children == 99:
        return 9
    else:
        raise ValueError('Invalid value in CHILDREN:', n_children)

In [10]:
df['CHILDREN_numeric'] = df['CHILDREN'].apply(convert_CHILDREN_numeric)
df['CHILDREN_categorical'] = df['CHILDREN'].apply(convert_CHILDREN_categorical).astype('category')

In [11]:
df[['CHILDREN', 'CHILDREN_numeric', 'CHILDREN_categorical']]

,CHILDREN,CHILDREN_numeric,CHILDREN_categorical
0,88.0,-1.0,8
1,88.0,-1.0,8
2,88.0,-1.0,8
3,88.0,-1.0,8
4,88.0,-1.0,8
...,...,...,...
401953,1.0,1.0,1
401954,4.0,4.0,1
401955,88.0,-1.0,8
401956,88.0,-1.0,8


In [12]:
def convert_HHADULT_numeric(n_adults):
    if 1 <= n_adults <= 76:
        return n_adults
    else:
        return -1

def convert_HHADULT_categorical(n_adults):
    if np.isnan(n_adults): # Not asked or missing
        return 0
    elif 1 <= n_adults <= 76:
        return 1
    elif n_adults == 77: # Don't know/Not sure
        return 7
    elif n_adults == 99: # Refused
        return 9
    else:
        raise ValueError('Invalid HHADULT:', n_adults)

In [13]:
df['HHADULT_numeric'] = df['HHADULT'].apply(convert_HHADULT_numeric)
df['HHADULT_categorical'] = df['HHADULT'].apply(convert_HHADULT_categorical).astype('category')

In [14]:
df[['HHADULT', 'HHADULT_numeric', 'HHADULT_categorical']]

,HHADULT,HHADULT_numeric,HHADULT_categorical
0,NaN,-1.0,0
1,NaN,-1.0,0
2,NaN,-1.0,0
3,NaN,-1.0,0
4,NaN,-1.0,0
...,...,...,...
401953,4.0,4.0,1
401954,2.0,2.0,1
401955,2.0,2.0,1
401956,2.0,2.0,1


In [15]:
def convert_HTM4_numeric(height):
    """Remains the valid height(cm) and convert empty to -1
    
    Orinally, the definition of HTM4 in the raw data is the height in
    "meter" rather than "centimeter". However, the valid height values in
    raw data are recorded in [91, 244] which is a range value in "centimeter".
    So, I decide to keep the original valid height in the raw data.
    """
    if 91 <= height <= 244:
        return height
    else:
        return -1

def convert_HTM4_categorical(height):
    if np.isnan(height):
        return 0
    elif 91 <= height <= 244:
        return 1
    else:
        raise ValueError('Invalid Value in HTM4:', height)

In [16]:
df['HTM4_numeric'] = df['HTM4'].apply(convert_HTM4_numeric)
df['HTM4_categorical'] = df['HTM4'].apply(convert_HTM4_categorical).astype('category')

In [17]:
df[['HTM4', 'HTM4_numeric', 'HTM4_categorical']]

,HTM4,HTM4_numeric,HTM4_categorical
0,170.0,170.0,1
1,163.0,163.0,1
2,173.0,173.0,1
3,NaN,-1.0,0
4,168.0,168.0,1
...,...,...,...
401953,150.0,150.0,1
401954,152.0,152.0,1
401955,124.0,124.0,1
401956,173.0,173.0,1


In [18]:
def convert_MARIJAN1_numeric(times):
    """Convert the frequency of using marijuana.
    
    Question:
    During the past 30 days, on how many days did you
    use marijuana or cannabis?
    
    Note:
    Although the person never used marijuana(大麻) is denoted "88",
    here, we convert the value to 0.
    """
    if 1 <= times <= 30:
        return times
    elif times == 88:
        return 0
    else:
        return -1

def convert_MARIJAN1_categorical(times):
    if np.isnan(times): # Not asked or missing
        return 0
    elif 1 <= times <= 30 or times == 88:
        return 1
    elif times == 77: # Don't know/Not sure
        return 7
    elif times == 99: # Refused
        return 9
    else:
        raise ValueError('Invalid value in MARIJAN1:', times)

In [19]:
df['MARIJAN1_numeric'] = df['MARIJAN1'].apply(convert_MARIJAN1_numeric)
df['MARIJAN1_categorical'] = df['MARIJAN1'].apply(convert_MARIJAN1_categorical).astype('category')

In [20]:
df[['MARIJAN1', 'MARIJAN1_numeric', 'MARIJAN1_categorical']]

,MARIJAN1,MARIJAN1_numeric,MARIJAN1_categorical
0,NaN,-1.0,0
1,NaN,-1.0,0
2,NaN,-1.0,0
3,NaN,-1.0,0
4,NaN,-1.0,0
...,...,...,...
401953,NaN,-1.0,0
401954,NaN,-1.0,0
401955,NaN,-1.0,0
401956,NaN,-1.0,0


In [21]:
def preprocess_HIVTSTD3_datetime(df):
    df['HIVTSTD3_month'] = df['HIVTSTD3'].apply(convert_HIVTSTD3_month).astype('category')
    df['HIVTSTD3_year']  = df['HIVTSTD3'].apply(convert_HIVTSTD3_year).astype('category')
    df['HIVTSTD3_categorical'] = df['HIVTSTD3'].apply(convert_HIVTSTD3_categorical).astype('category')
    return df

def check_and_convert_int(month_year):
    if not isinstance(month_year, int):
        month_year = int(month_year)
    return month_year

def convert_HIVTSTD3_month(month_year):
    """
    Note:
    Corner Error example: missing month (already implemented)
    - HIVTSTD3: 772019
    - HIVTSTD3: 772017
    - HIVTSTD3: 772005
    """
    if np.isnan(month_year):
        return -1
    
    month_year = check_and_convert_int(month_year)
    
    month_year = str(month_year)
    if len(month_year) == 5:
        month = int(month_year[0])
        return month if 1 <= month <= 9 else -1
    elif len(month_year) == 6:
        month = int(month_year[:1])
        return month if 10 <= month <= 12 else -1
    else:
        raise ValueError(invalid_col_val_msg('HIVTSTD3', month_year,\
                         additional=' (converting month error)'))

def convert_HIVTSTD3_year(month_year):
    if np.isnan(month_year):
        return -1
    
    month_year = check_and_convert_int(month_year)
    
    month_year = str(month_year)
    if len(month_year) == 5:
        year = int(month_year[1:])
        return year if 1985 <= year <= 2021 else -1
    elif len(month_year) == 6:
        year = int(month_year[2:])
        return year if 1985 <= year <= 2021 else -1
    else:
        raise ValueError(invalid_col_val_msg('HIVTSTD3', month_year,\
                         additional=' (converting year error)'))

def convert_HIVTSTD3_categorical(month_year):
    """
    Note:
    Corner Error example: missing month (already implemented)
    - HIVTSTD3: 772019
    - HIVTSTD3: 772017
    - HIVTSTD3: 772005
    """
    if np.isnan(month_year):
        return 0
   
    month_year = check_and_convert_int(month_year)
    if 11985 <= month_year <= 122021:
        return 1
    elif month_year == 777777 or 771980 <= month_year <= 772021:
        return 7
    elif month_year == 999999:
        return 9
    else:
        raise ValueError(invalid_col_val_msg('HIVTSTD3', month_year))

In [22]:
preprocess_HIVTSTD3_datetime(df)

,Unnamed: 0,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,...,CHILDREN_categorical,HHADULT_numeric,HHADULT_categorical,HTM4_numeric,HTM4_categorical,MARIJAN1_numeric,MARIJAN1_categorical,HIVTSTD3_month,HIVTSTD3_year,HIVTSTD3_categorical
0,0,1.0,1.0,b'01042020',b'01',b'04',b'2020',1100.0,b'2020000001',2.020000e+09,...,8,-1.0,0,170.0,1,-1.0,0,2,1994,1
1,1,1.0,1.0,b'02072020',b'02',b'07',b'2020',1200.0,b'2020000002',2.020000e+09,...,8,-1.0,0,163.0,1,-1.0,0,-1,-1,0
2,2,1.0,1.0,b'01232020',b'01',b'23',b'2020',1100.0,b'2020000003',2.020000e+09,...,8,-1.0,0,173.0,1,-1.0,0,-1,-1,0
3,3,1.0,1.0,b'01092020',b'01',b'09',b'2020',1100.0,b'2020000004',2.020000e+09,...,8,-1.0,0,-1.0,0,-1.0,0,-1,-1,0
4,4,1.0,1.0,b'01042020',b'01',b'04',b'2020',1100.0,b'2020000005',2.020000e+09,...,8,-1.0,0,168.0,1,-1.0,0,-1,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401953,401953,72.0,11.0,b'02192021',b'02',b'19',b'2021',1100.0,b'2020004940',2.020005e+09,...,1,4.0,1,150.0,1,-1.0,0,-1,-1,7
401954,401954,72.0,11.0,b'02142021',b'02',b'14',b'2021',1100.0,b'2020004941',2.020005e+09,...,1,2.0,1,152.0,1,-1.0,0,9,2019,1
401955,401955,72.0,11.0,b'02142021',b'02',b'14',b'2021',1100.0,b'2020004942',2.020005e+09,...,8,2.0,1,124.0,1,-1.0,0,-1,-1,0
401956,401956,72.0,11.0,b'03172021',b'03',b'17',b'2021',1100.0,b'2020004943',2.020005e+09,...,8,2.0,1,173.0,1,-1.0,0,-1,-1,7


In [23]:
df[['HIVTSTD3', 'HIVTSTD3_year', 'HIVTSTD3_month', 'HIVTSTD3_categorical']]

,HIVTSTD3,HIVTSTD3_year,HIVTSTD3_month,HIVTSTD3_categorical
0,21994.0,1994,2,1
1,NaN,-1,-1,0
2,NaN,-1,-1,0
3,NaN,-1,-1,0
4,NaN,-1,-1,0
...,...,...,...,...
401953,777777.0,-1,-1,7
401954,92019.0,2019,9,1
401955,NaN,-1,-1,0
401956,777777.0,-1,-1,7
